# Cleaning salary survey data

This data comes from [Ask A Manager's annual survey for 2023](https://www.askamanager.org/2023/04/how-much-money-do-you-make-6.html). The spreadsheet is available [here](https://docs.google.com/spreadsheets/d/1ioUjhnz6ywSpEbARI-G3RoPyO0NRBqrJnWf-7C_eirs/edit?resourcekey#gid=1854892322). I pulled the data on August 13th, so data pulled at a later date will probably contain more rows.

I found this source thanks to [Oscar Baruffa](https://oscarbaruffa.com/messy/), and as he points out, the dataset is great for cleaning because it's:
* Real-life
* Both numeric and text, where some text columns are categorical and others are free-form
* Full of nulls

Knowing the use case for a dataset is key to making correct decisions during cleaning: which columns to keep, which to modify, and how to make the end user's job easier. For this project, I'll clean the data as if a data analyst were the end user.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Salary_Survey_2023.csv")

## Basic exploration

In [3]:
df.head()

,Timestamp,How old are you?,Industry,Functional area of job,Job title,Job title - additional context,Annual salary (gross),Additional monetary compensation,Currency,Currency - other,Income - additional context,Country,State,City,Remote or on-site?,"Years of experience, overall",Years of experience in field,Highest level of education completed,Gender,Race
0,4/11/2023 11:02:00,35-44,Government & Public Administration,Engineering or Manufacturing,Materials Engineer,NaN,125000,800.0,USD,NaN,NaN,United States,California,Ridgecrest,On-site,11-20 years,11-20 years,College degree,Man,White
1,4/11/2023 11:02:07,25-34,"Galleries, Libraries, Archives & Museums","Galleries, Libraries, Archives & Museums",Assistant Branch Manager,NaN,71000,0.0,USD,NaN,NaN,United States,Virginia,Fairfax County,On-site,8-10 years,5-7 years,Master's degree,Man,White
2,4/11/2023 11:02:12,35-44,Education (Higher Education),Education (Higher Education),Director of Financial Aid,NaN,60000,0.0,USD,NaN,NaN,United States,Oklahoma,Anadarko,On-site,21-30 years,11-20 years,College degree,Woman,White
3,4/11/2023 11:02:15,25-34,Education (Higher Education),Government & Public Administration,Administrative Assistant,NaN,42000,NaN,USD,NaN,NaN,United States,Virginia,Richmond,On-site,2-4 years,2-4 years,College degree,Man,White
4,4/11/2023 11:02:25,18-24,"Accounting, Banking & Finance",Administration,Executive Assistant,NaN,65000,0.0,USD,NaN,NaN,United States,Utah,Orem,On-site,2-4 years,2-4 years,Some college,Woman,White


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16898 entries, 0 to 16897
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Timestamp                             16898 non-null  object 
 1   How old are you?                      16898 non-null  object 
 2   Industry                              16852 non-null  object 
 3   Functional area of job                16787 non-null  object 
 4   Job title                             16898 non-null  object 
 5   Job title - additional context        3841 non-null   object 
 6   Annual salary (gross)                 16898 non-null  int64  
 7   Additional monetary compensation      12952 non-null  float64
 8   Currency                              16898 non-null  object 
 9   Currency - other                      89 non-null     object 
 10  Income - additional context           1671 non-null   object 
 11  Country        

In [5]:
df["Race"].value_counts().tail()

Race
Asian or Asian American, Black or African American, Native American or Alaska Native, White                                                                      1
Asian or Asian American, Hispanic, Latino, or Spanish origin, Native American or Alaska Native, White                                                            1
Asian or Asian American, Hispanic, Latino, or Spanish origin, Native American or Alaska Native, White, Another option not listed here or prefer not to answer    1
Asian or Asian American, Black or African American, Hispanic, Latino, or Spanish origin, White                                                                   1
Middle Eastern or Northern African, White, Another option not listed here or prefer not to answer                                                                1
Name: count, dtype: int64

Some observations:
* Some columns that we'd expect to be numbers are actually text ("Years of experience in field"; ranges instead of integers for "How old are you?" and years of experience). 
* Some have many NaN values ("Job title - additional context", "Currency - other", "Income - additional context"). For "Additional monetary compensation", a 0 value and a NaN value seem to have the same meaning, but respondents have entered both.
* Some columns, such as "Race", had the option to select multiple values.
* Many of the columns have long, unwieldy names.

In addition, some columns that respondents could enter free-form have unexpected values, such as "Fairfax County" in the "City" column for row 1. In a production context, I would speak to a subject matter expert to determine whether to drop the affected rows, alter their values, or keep them as-is. In this case I will keep the values as-is to avoid loss of information.

## Drop unneeded data

First, to make sure no accidental duplicates exist:

In [6]:
df.drop_duplicates(inplace=True)

Assuming that the end user doesn't care when responses were submitted, let"s remove the Timestamp column.

In [7]:
df.drop(columns="Timestamp", inplace=True)

The "Job title" column is freeform. It's possible that every entry is unique. This field isn't as useful for analysis, so I will drop it. I will also drop two similar columns: "Job title - additional context" and "Income - additional context".

In [8]:
df.drop(columns=["Job title", "Job title - additional context", "Income - additional context"], inplace=True)

## Rename columns

Some of the column names are long. I'll rename them so they are easier to type.

In [9]:
df.rename(columns={"How old are you?": "Age",
                   "Functional area of job": "Job function",
                   "Annual salary (gross)": "Salary",
                   "Additional monetary compensation": "Additional compensation",
                   "Remote or on-site?": "Work location",
                   "Years of experience, overall": "Experience overall",
                   "Years of experience in field": "Experience in field",
                   "Highest level of education completed": "Education"}, inplace=True)

In [10]:
list(df.columns)

['Age',
 'Industry',
 'Job function',
 'Salary',
 'Additional compensation',
 'Currency',
 'Currency - other',
 'Country',
 'State',
 'City',
 'Work location',
 'Experience overall',
 'Experience in field',
 'Education',
 'Gender',
 'Race']

## Convert data types

In [11]:
df.dtypes

Age                         object
Industry                    object
Job function                object
Salary                       int64
Additional compensation    float64
Currency                    object
Currency - other            object
Country                     object
State                       object
City                        object
Work location               object
Experience overall          object
Experience in field         object
Education                   object
Gender                      object
Race                        object
dtype: object

In [12]:
df["Age"].value_counts(sort=False)

Age
35-44         6937
25-34         5885
18-24          342
45-54         2726
55-64          901
65 or over     104
under 18         3
Name: count, dtype: int64

In [19]:
# ['Age', make categorical
# 'Industry', make categorical
#'Job function', make categorical
# Salary convert all to USD. Check for outliers?
# Additional compensation convert all to USD
#' 'Currency', drop once convert to USD
#'Currency - other', same as above
#'Country',  clean to extract/standardize. once done, make categorical?
#'State', deal with multivalues - get_dummies()?
#'City', clean to remove state names?
#'Work location', make categorical
#'Experience overall', make categorical
#'Experience in field', make categorical
#'Education', clean to deal with 'other' inputs then make categorical
#'Gender', get_dummies() 
#'Race'] get_dummies()
 

In [18]:
df['Education'].value_counts()

Education
College degree                                                             7627
Master's degree                                                            5649
Some college                                                               1149
PhD                                                                        1081
Professional degree (MD, JD, etc.)                                          882
                                                                           ... 
CPA                                                                           1
EdD in progress                                                               1
Postgraduate Diploma in Science                                               1
Bachelor's Degree, from a University (Colleges are different in Canada)       1
Associates Degree in Social & Behavioral Sciences                             1
Name: count, Length: 182, dtype: int64

The purely numeric columns, Salary and Additional compensation, already have numeric dtypes. 
Several other columns can be converted to the Categorical dtype, [which saves memory when there are far fewer categories than rows](https://pandas.pydata.org/docs/user_guide/categorical.html#categorical-memory). A Categorical type also allows user-defined ordering where desired (so that, for example, "under 18" can come before "18-24" in the Age column).

In [16]:
from pandas.api.types import CategoricalDtype

In [17]:
age_type = CategoricalDtype(categories=["under 18",
                                        "18-24",
                                        "25-34",
                                        "35-44",
                                        "45-54",
                                        "55-64",
                                        "65 or over"], ordered=True)

df["Age"] = df["Age"].astype(age_type)

In [18]:
experience_type = CategoricalDtype(categories=["1 year or less",
                                                "2-4 years",
                                                "5-7 years",
                                                "8-10 years",
                                                "11-20 years",
                                                "21-30 years",
                                                "31-40 years",
                                                "41 years or more"], ordered=True)

df["Experience overall"] = df["Experience overall"].astype(experience_type)
df["Experience in field"] = df["Experience in field"].astype(experience_type)

In [21]:
df["Education"].value_counts()

Education
College degree                                                             7627
Master's degree                                                            5649
Some college                                                               1149
PhD                                                                        1081
Professional degree (MD, JD, etc.)                                          882
                                                                           ... 
CPA                                                                           1
EdD in progress                                                               1
Postgraduate Diploma in Science                                               1
Bachelor's Degree, from a University (Colleges are different in Canada)       1
Associates Degree in Social & Behavioral Sciences                             1
Name: count, Length: 182, dtype: int64

In [22]:
category_columns = ["Industry", "Job function", "Country", "State", "Work location"]
for c in category_columns:
    df[c] = df[c].astype("category")

## Standardize income

For consistency, let's replace NaN values with 0 in the "Additional compensation" column.

In [27]:
df["Additional compensation"].fillna(0, inplace=True)

Standardize amounts by converting all non-USD values to USD.

In [ ]:
#remove currency cols

For easier querying, create a column of total income (salary plus additional compensation).

## The complete script - short version